<a href="https://colab.research.google.com/github/JamesMartinOU/PublicRedditSentimentAnalysis/blob/main/RedditPostsToDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step One - Python libraries and Global variables.

1.   Install Python libraries
2.   Load Python libraries
3.   Define Global variables



In [ ]:
# Install Python libraries
!pip install praw
!pip install pandas
!pip install mysql-connector-python

In [ ]:
# Load Python libraries
import pandas as pd
import mysql.connector
import praw
from google.colab import files
import time
import prawcore

In [ ]:
# Define dictionary containing Top 100 Companies, CEOs, and Stock Symbol
fortune_100_ceos_with_symbols = [
    {"Company": "Walmart", "CEO": "Doug McMillon", "Stock Symbol": "WMT"},
    {"Company": "Amazon", "CEO": "Andy Jassy", "Stock Symbol": "AMZN"},
    {"Company": "State Grid", "CEO": "Kang Yong", "Stock Symbol": "SGRE"},
    {"Company": "Saudi Aramco", "CEO": "Amin H. Nasser", "Stock Symbol": "2222.SR"},
    {"Company": "Sinopec", "CEO": "Dai Houliang", "Stock Symbol": "0386.HK"},
    {"Company": "Apple", "CEO": "Tim Cook", "Stock Symbol": "AAPL"},
    {"Company": "UnitedHealth Group", "CEO": "Andrew Witty", "Stock Symbol": "UNH"},
    {"Company": "Berkshire Hathaway", "CEO": "Warren Buffett", "Stock Symbol": "BRK.A"},
    {"Company": "CVS Health", "CEO": "Karen S. Lynch", "Stock Symbol": "CVS"},
    {"Company": "Exxon Mobil", "CEO": "Darren W. Woods", "Stock Symbol": "XOM"},
    {"Company": "McKesson", "CEO": "Brian Tyler", "Stock Symbol": "MCK"},
    {"Company": "Cencora", "CEO": "Everett L. (Ed) Schoonover", "Stock Symbol": "CNC"},
    {"Company": "Cigna", "CEO": "David Cordani", "Stock Symbol": "CI"},
    {"Company": "Cardinal Health", "CEO": "Mike Kaufmann", "Stock Symbol": "CAH"},
    {"Company": "Comcast", "CEO": "Brian L. Roberts", "Stock Symbol": "CMCSA"},
    {"Company": "Costco Wholesale", "CEO": "Craig Jelinek", "Stock Symbol": "COST"},
    {"Company": "Tesla", "CEO": "Elon Musk", "Stock Symbol": "TSLA"},
    {"Company": "Walgreens Boots Alliance", "CEO": "Rosalind Brewer", "Stock Symbol": "WBA"},
    {"Company": "Verizon Communications", "CEO": "Hans Vestberg", "Stock Symbol": "VZ"},
    {"Company": "AmerisourceBergen", "CEO": "Steven H. Collis", "Stock Symbol": "ABC"},
    {"Company": "Chevron", "CEO": "Mike Wirth", "Stock Symbol": "CVX"},
    {"Company": "Home Depot", "CEO": "Craig Menear", "Stock Symbol": "HD"},
    {"Company": "Ford Motor", "CEO": "Jim Farley", "Stock Symbol": "F"},
    {"Company": "Caterpillar", "CEO": "Jim Umpleby", "Stock Symbol": "CAT"},
    {"Company": "Microsoft", "CEO": "Satya Nadella", "Stock Symbol": "MSFT"},
    {"Company": "Kroger", "CEO": "Rodney McMullen", "Stock Symbol": "KR"},
    {"Company": "Marathon Petroleum", "CEO": "Michael J. Hennigan", "Stock Symbol": "MPC"},
    {"Company": "General Motors", "CEO": "Mary Barra", "Stock Symbol": "GM"},
    {"Company": "PepsiCo", "CEO": "Ramon Laguarta", "Stock Symbol": "PEP"},
    {"Company": "Johnson & Johnson", "CEO": "Joaquin Duato", "Stock Symbol": "JNJ"},
    {"Company": "HP", "CEO": "Enrique Lores", "Stock Symbol": "HPQ"},
    {"Company": "UPS", "CEO": "Carol B. Tomé", "Stock Symbol": "UPS"},
    {"Company": "Hewlett Packard Enterprise", "CEO": "Antonio Neri", "Stock Symbol": "HPE"},
    {"Company": "Anthem", "CEO": "Gail Boudreaux", "Stock Symbol": "ANTM"},
    {"Company": "Bristol-Myers Squibb", "CEO": "Giovanni Caforio", "Stock Symbol": "BMY"},
    {"Company": "Target", "CEO": "Brian Cornell", "Stock Symbol": "TGT"},
    {"Company": "Pfizer", "CEO": "Albert Bourla", "Stock Symbol": "PFE"},
    {"Company": "Archer Daniels Midland", "CEO": "Juan Luciano", "Stock Symbol": "ADM"},
    {"Company": "Procter & Gamble", "CEO": "Jon Moeller", "Stock Symbol": "PG"},
    {"Company": "Morgan Stanley", "CEO": "James P. Gorman", "Stock Symbol": "MS"},
    {"Company": "Best Buy", "CEO": "Corie Barry", "Stock Symbol": "BBY"},
    {"Company": "Lockheed Martin", "CEO": "James D. Taiclet", "Stock Symbol": "LMT"},
    {"Company": "T-Mobile US", "CEO": "Mike Sievert", "Stock Symbol": "TMUS"},
    {"Company": "AbbVie", "CEO": "Richard A. Gonzalez", "Stock Symbol": "ABBV"},
    {"Company": "Lowe's", "CEO": "Marvin R. Ellison", "Stock Symbol": "LOW"},
    {"Company": "Raytheon Technologies", "CEO": "Gregory J. Hayes", "Stock Symbol": "RTX"},
    {"Company": "Intel", "CEO": "Patrick Gelsinger", "Stock Symbol": "INTC"},
    {"Company": "Hilton Worldwide", "CEO": "Christopher J. Nassetta", "Stock Symbol": "HLT"},
    {"Company": "Oracle", "CEO": "Safra Catz", "Stock Symbol": "ORCL"},
    {"Company": "AIG", "CEO": "Peter Zaffino", "Stock Symbol": "AIG"},
    {"Company": "Wells Fargo", "CEO": "Charles W. Scharf", "Stock Symbol": "WFC"},
    {"Company": "Sysco", "CEO": "Kevin Hourican", "Stock Symbol": "SYY"},
    {"Company": "General Electric", "CEO": "H. Lawrence Culp Jr.", "Stock Symbol": "GE"},
    {"Company": "The Coca-Cola Company", "CEO": "James Quincey", "Stock Symbol": "KO"},
    {"Company": "IBM", "CEO": "Arvind Krishna", "Stock Symbol": "IBM"},
    {"Company": "State Farm Insurance", "CEO": "Michael L. Tipsord", "Stock Symbol": "STT"},
    {"Company": "MetLife", "CEO": "Michel A. Khalaf", "Stock Symbol": "MET"},
    {"Company": "Honeywell International", "CEO": "Darius Adamczyk", "Stock Symbol": "HON"},
    {"Company": "Delta Air Lines", "CEO": "Ed Bastian", "Stock Symbol": "DAL"},
    {"Company": "AT&T", "CEO": "John Stankey", "Stock Symbol": "T"},
    {"Company": "HCA Healthcare", "CEO": "Sam Hazen", "Stock Symbol": "HCA"},
    {"Company": "New York Life Insurance", "CEO": "Ted Mathas", "Stock Symbol": "NYL"},
    {"Company": "Discovery, Inc.", "CEO": "David Zaslav", "Stock Symbol": "DISCA"},
    {"Company": "BlackRock", "CEO": "Larry Fink", "Stock Symbol": "BLK"},
    {"Company": "Fannie Mae", "CEO": "Hugh R. Frater", "Stock Symbol": "FNMA"},
    {"Company": "The Walt Disney Company", "CEO": "Bob Chapek", "Stock Symbol": "DIS"},
    {"Company": "Aetna", "CEO": "Mark T. Bertolini", "Stock Symbol": "AET"},
    {"Company": "Morgan Stanley", "CEO": "James P. Gorman", "Stock Symbol": "MS"}
]


# Step Two - Define Python functions to fetch Reddit posts and code block to write Reddit posts.
1.   fetch_reddit_posts fuction - to write Reddit posts to dataframe.
2.   Write all posts to csv file.



In [ ]:
# Reddit Posts function
def fetch_reddit_posts(query, limit=return_limit):
    posts_data = []
    try:
        for post in reddit.subreddit("all").search(query, sort='top', limit=limit):
          posts_data.append({
                "id": post.id,
                "title": post.title,
                "subreddit": post.subreddit.display_name,
                "created_utc": post.created_utc,
                "url": post.url,
                "score": post.score,
                "upvote_ratio": post.upvote_ratio,
                "num_comments": post.num_comments,
                "author": post.author.name if post.author else None,  # Author's username
                "author_fullname": post.author_fullname if post.author else None,
                "num_crossposts": post.num_crossposts,
                "keyword": query,  # Add the keyword to the post data
                "company_name": company_name  # Add the tied company name
            })

        # Convert to DataFrame
        df = pd.DataFrame(posts_data)
        return df
    except Exception as e:
        print(f"Error fetching posts: {e}")
        return pd.DataFrame()


In [ ]:
# Write posts to CSV and all_posts_df
# Initialize an empty DataFrame to store all posts
all_posts_df = pd.DataFrame()

# Fetch posts for each keyword and append to all_posts_df
for company in fortune_100_ceos_with_symbols:
    company_name = company["Company"]
    ceo_name = company["CEO"]
    stock_symbol = company["Stock Symbol"]

    df_reddit_posts = fetch_reddit_posts(company_name, limit=return_limit)
    all_posts_df = pd.concat([all_posts_df, df_reddit_posts], ignore_index=True)  # Append the new company name results

    df_reddit_posts = fetch_reddit_posts(ceo_name, limit=return_limit)
    all_posts_df = pd.concat([all_posts_df, df_reddit_posts], ignore_index=True)  # Append the new ceo results

    df_reddit_posts = fetch_reddit_posts(stock_symbol, limit=return_limit)
    all_posts_df = pd.concat([all_posts_df, df_reddit_posts], ignore_index=True)  # Append the new stock

    time.sleep(2)  # Sleep to avoid rate limits

# Data cleansing to remove duplicate records and those missing url then convert utc (unix) to date time
all_posts_df = all_posts_df.drop_duplicates()
all_posts_df = all_posts_df.drop_duplicates(subset='id', keep='first')
all_post_df = df_reddit_posts[df_reddit_posts['url'].str.len() >= 5]


# Save the combined DataFrame to CSV
csv_file_name = "reddit_posts.csv"
all_posts_df.to_csv(csv_file_name, index=False)

# Download the CSV file
files.download(csv_file_name)


# Step Three - Write all_posts_df to MySQL table reddit_posts.

In [ ]:
# Write reddit posts to MySQL table
try:
    # Establish connection to MySQL
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME
    )

    cursor = conn.cursor()

    # Insert data row by row (You can optimize this with batch insert if needed)
    for _, row in all_posts_df.iterrows():
        created_utc_mysql = pd.to_datetime(row['created_utc'], unit='s').strftime('%Y-%m-%d %H:%M:%S')
        cursor.execute("""
            INSERT INTO reddit_posts (id, title, subreddit, created_utc, url, score, upvote_ratio, num_comments, author, author_fullname, num_crossposts, keyword, company_name)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['id'],
            row['title'],
            row['subreddit'],
            created_utc_mysql,
            row['url'],
            row['score'],
            row['upvote_ratio'],
            row['num_comments'],
            row['author'],
            row['author_fullname'],
            row['num_crossposts'],
            row['keyword'],
            row['company_name']
        ))

    # Commit the transaction
    conn.commit()
    print("✅ Data inserted successfully into the MySQL database.")

except mysql.connector.Error as err:
    print(f"❌ Error: {err}")

finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print("🔌 Connection closed.")